# **Assignment No-3 Register and manage trained models using MLflow Model Registry or AWS SageMaker **

Setup

In [1]:
!pip install mlflow
!pip install pyngrok

import mlflow
import mlflow.sklearn
from pyngrok import ngrok



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import Libraries

In [2]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Generate Sample DATA

In [3]:
X, y = make_regression(n_samples=100, n_features=1, noise=10)
X_train, X_test, y_train, y_test = train_test_split(X, y)


# Start MLflow Tracking Server

In [2]:
!pip install mlflow


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# -------------------------------
# SET TRACKING URI (your MLflow server)
# -------------------------------
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Localhost MLflow UI

# -------------------------------
# LOAD DATA
# -------------------------------
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42
)

# -------------------------------
# START AN EXPERIMENT
# -------------------------------
mlflow.set_experiment("Iris_RF_Experiment")

with mlflow.start_run(run_name="RandomForest_Model_v1"):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)

    # Log parameters, metrics, and model
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("accuracy", acc)
    mlflow.sklearn.log_model(model, "model")

    print("✅ Model logged with accuracy:", acc)


2025/11/06 00:56:26 INFO mlflow.tracking.fluent: Experiment with name 'Iris_RF_Experiment' does not exist. Creating a new experiment.
2025/11/06 00:56:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/06 00:57:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Model logged with accuracy: 1.0
🏃 View run RandomForest_Model_v1 at: http://127.0.0.1:5000/#/experiments/928544864268208532/runs/4fbc9489e52d471492ad1a5c06553fad
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/928544864268208532


In [4]:
# --------------------------------------------------------
# STEP 4 — Register the Model in MLflow Model Registry
# --------------------------------------------------------

# Get the last run ID (or you can store it inside the run block)
run = mlflow.last_active_run()
model_uri = f"runs:/{run.info.run_id}/model"

# Register model in the registry
registered_model = mlflow.register_model(
    model_uri=model_uri,
    name="Iris_RandomForest_Model"  # your model’s registry name
)

print(f"✅ Model registered successfully!")
print(f"Model name: {registered_model.name}")
print(f"Version: {registered_model.version}")


Successfully registered model 'Iris_RandomForest_Model'.
2025/11/06 01:06:13 WARNING mlflow.tracking._model_registry.fluent: Run with id 4fbc9489e52d471492ad1a5c06553fad has no artifacts at artifact path 'model', registering model based on models:/m-11f0f2e47a594981b351cae61d1628db instead
2025/11/06 01:06:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Iris_RandomForest_Model, version 1


✅ Model registered successfully!
Model name: Iris_RandomForest_Model
Version: 1


Created version '1' of model 'Iris_RandomForest_Model'.


In [5]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.transition_model_version_stage(
    name="Iris_RandomForest_Model",
    version=1,
    stage="Staging"  # or "Production"
)


C:\Users\rutuj\AppData\Local\Temp\ipykernel_20764\3409569993.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1762371373305, current_stage='Staging', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1762371626082, metrics=None, model_id=None, name='Iris_RandomForest_Model', params=None, run_id='4fbc9489e52d471492ad1a5c06553fad', run_link='', source='models:/m-11f0f2e47a594981b351cae61d1628db', status='READY', status_message=None, tags={}, user_id='', version='1'>